# This notebook was created to 1. collect (SNP & INDEL) variants in *eis* and the *whiB7* promoter and 2. detect instances of co-occurence of LoF variants in *eis* and the *whiB7* promoter

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import vcf

%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from itertools import compress
from pylab import MaxNLocator
import seaborn as sns; sns.set()
from matplotlib.colors import LogNorm
from matplotlib import gridspec
import ast
import itertools
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import fastcluster
from sklearn import cluster, datasets
import scipy.cluster.hierarchy as hier
from sklearn.cluster import KMeans
import time
import sys
import pickle

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC
from Bio import Phylo

from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq
import itertools
import gzip

import networkx as nx
import scipy
from collections import Counter

# *whiB7* promoter & *eis*

promoter (upstream of *whiB7*): 3,568,680 - 3,569,082

coding (*eis*): 2,714,124 - 2,715,332

H37Rv region: 2,714,124 - 3,569,082

In [5]:
regulator_id = 'Rv3197A_Rv3198c'
H37Rv_region_start_A = 3568680
H37Rv_region_end_A = 3569082

regulon_id = 'Rv2416c'
H37Rv_region_start_B = 2714124
H37Rv_region_end_B = 2715332

### *Function* to convert lineage-calls to new/simpler lineage call scheme

In [6]:
def convert_lineage_calls(isolate_annotation_DF):

    lineage_hierarchincal_to_lineage_map_df = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/CSV files/lineage_hierachical_to_lineage_map.csv')
    lineage_hierarchincal_to_lineage_map_series = pd.Series(list(lineage_hierarchincal_to_lineage_map_df.loc[:, 'lineage']), index = lineage_hierarchincal_to_lineage_map_df.lineage_hierarchical)

    lineage_call_update_list = []

    for hierarchical_lineage_call in isolate_annotation_DF.lineage_call:

        # if lineage call has an updated shorter call
        if hierarchical_lineage_call in lineage_hierarchincal_to_lineage_map_series.index:
            lineage_call_update_list.append(lineage_hierarchincal_to_lineage_map_series[hierarchical_lineage_call])

        # if lineage call doesn't have an updated shorter call
        else:
            lineage_call_update_list.append(hierarchical_lineage_call)

    # replace the hierarchical calls with the shorter ones
    isolate_annotation_DF.loc[:,'lineage_call'] = lineage_call_update_list
    
    return isolate_annotation_DF

# [1] INDELs

## [1.1] Load INDEL genotype matrix and Annotation Files

### 31,428 isolates

In [7]:
#load isolate annotation file (columns of Genotype Matrix)
INDEL_isolate_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape_indels/Genotypes_Filtered_2/genotypes_isolate_annotation.pkl')
INDEL_isolate_annotation_DF = INDEL_isolate_annotation_DF.loc[:, ['isolate_ID','lineage_call']] #drop columns
INDEL_isolate_annotation_DF = convert_lineage_calls(INDEL_isolate_annotation_DF)

#load INDEL annotation file (rows of Genotype Matrix) with gene annotation information
INDEL_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape_indels/Genotypes_Filtered_2/genotypes_INDEL_functional_annotation.pkl')
INDEL_annotation_DF.reset_index(inplace = True, drop = False)

#load Genotypes Matrix
INDEL_genotypes_array =  np.load('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape_indels/Genotypes_Filtered_2/genotypes_matrix.npy')

In [8]:
INDEL_isolate_annotation_DF.head()

,isolate_ID,lineage_call
0,SAMN13051687,2.2.1.1.1.i3
1,SAMN09100245,4.3.i4.1
2,SAMN08732238,2.2.1.1.1
3,SAMN07658260,3.1.1
4,SAMN03648003,2.2.1.1.1


In [9]:
np.shape(INDEL_isolate_annotation_DF)

(31428, 2)

In [10]:
INDEL_annotation_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,ins_del,INDEL_type,codon_pos
0,ACCGACGAAG_313_A,313,ACCGACGAAG,A,Essential,dnaA,Rv0001,313,del,inframe,105
1,TC_1549_T,1549,TC,T,None,None,Rv0001_Rv0002,None,del,frameshift,None
2,T_1552_TAA,1552,T,TAA,None,None,Rv0001_Rv0002,None,ins,frameshift,None
3,TAA_1552_T,1552,TAA,T,None,None,Rv0001_Rv0002,None,del,frameshift,None
4,T_1552_TA,1552,T,TA,None,None,Rv0001_Rv0002,None,ins,frameshift,None


In [11]:
np.shape(INDEL_annotation_DF)

(50260, 11)

In [12]:
INDEL_genotypes_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [13]:
np.shape(INDEL_genotypes_array)

(50260, 31428)

### additional 12 eis C-14T mutants with AG MICs

In [14]:
#load isolate annotation file (columns of Genotype Matrix)
isolate_annotation_DF_extra_strains = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/eis_promoter_mutant_strains_from_Milan/Genotypes/Genotypes_Filtered/genotypes_isolate_annotation.pkl')
isolate_annotation_DF_extra_strains = isolate_annotation_DF_extra_strains.loc[:, ['isolate_ID','lineage_call']]
isolate_annotation_DF_extra_strains = convert_lineage_calls(isolate_annotation_DF_extra_strains)

#load Genotypes Matrix
genotypes_array_extra_strains =  np.load('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/eis_promoter_mutant_strains_from_Milan/Genotypes_indels/Genotypes_Filtered/genotypes_matrix.npy')

In [15]:
isolate_annotation_DF_extra_strains.head()

,isolate_ID,lineage_call
0,168-19,2.2.1.1.1
1,622-19,2.2.1.1.1.i3
2,655-19,4.3.i3.1
3,IT1070,2.2.1.1.1.i3
4,IT123,2.2.1.1.1


In [16]:
genotypes_array_extra_strains

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 9, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [17]:
np.shape(genotypes_array_extra_strains)

(50260, 12)

### Merge *isolate annotation files* and *genotypes matrices* together

In [18]:
# merge isolate annotation dataframes
INDEL_isolate_annotation_DF = INDEL_isolate_annotation_DF.append(isolate_annotation_DF_extra_strains)
INDEL_isolate_annotation_DF.reset_index(inplace = True, drop = True)

In [19]:
np.shape(INDEL_isolate_annotation_DF)

(31440, 2)

In [20]:
# merge genotypes matrices
INDEL_genotypes_array = np.hstack((INDEL_genotypes_array, genotypes_array_extra_strains))

In [21]:
np.shape(INDEL_genotypes_array)

(50260, 31440)

## [1.2] Subset to targeted chomosomal region

In [22]:
# keep only indels that occur within chromosomal region
indels_in_targeted_region = list(INDEL_annotation_DF[((INDEL_annotation_DF.pos >= H37Rv_region_start_A) & (INDEL_annotation_DF.pos <= H37Rv_region_end_A)) | ((INDEL_annotation_DF.pos >= H37Rv_region_start_B) & (INDEL_annotation_DF.pos <= H37Rv_region_end_B))].key)

# construct boolean filter
indels_to_keep_filter = [indel_i in indels_in_targeted_region for indel_i in INDEL_annotation_DF.key]

# filter Genotypes Matrix
INDEL_genotypes_array = INDEL_genotypes_array[indels_to_keep_filter , :]

# filter INDEL annotation file
INDEL_annotation_DF = INDEL_annotation_DF[indels_to_keep_filter]
INDEL_annotation_DF.reset_index(drop = True , inplace = True) #re-index new filtered INDEL annotation DF (so new index matches indexing of genotypes matrix rows)

In [29]:
INDEL_annotation_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,ins_del,INDEL_type,codon_pos
0,A_2714157_ACTG,2714157,A,ACTG,Non-Essential,eis,Rv2416c,1176,ins,inframe,392
1,T_2714272_TC,2714272,T,TC,Non-Essential,eis,Rv2416c,1061,ins,frameshift,354
2,C_2714303_CG,2714303,C,CG,Non-Essential,eis,Rv2416c,1030,ins,frameshift,344
3,G_2714334_GA,2714334,G,GA,Non-Essential,eis,Rv2416c,999,ins,frameshift,333
4,ACTCG_2714384_A,2714384,ACTCG,A,Non-Essential,eis,Rv2416c,949,del,frameshift,317


In [25]:
np.shape(INDEL_annotation_DF)

(40, 11)

In [26]:
INDEL_genotypes_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [27]:
np.shape(INDEL_genotypes_array)

(40, 31440)

# [2] SNPs

### 31,428 isolates

## [2.1] Load SNP genotype matrix and Annotation Files

In [30]:
#load isolate annotation file (columns of Genotype Matrix)
SNP_isolate_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_isolate_annotation.pkl')
SNP_isolate_annotation_DF = SNP_isolate_annotation_DF.loc[:, ['isolate_ID','lineage_call']] # drop columns
SNP_isolate_annotation_DF = convert_lineage_calls(SNP_isolate_annotation_DF)

#load INDEL annotation file (rows of Genotype Matrix) with gene annotation information
SNP_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl')
SNP_annotation_DF = SNP_annotation_DF.loc[:, ['pos','ref','alt']] # drop columns
SNP_annotation_DF.reset_index(inplace = True, drop = False)

#load Genotypes Matrix
SNP_genotypes_array =  np.load('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy')

In [31]:
SNP_isolate_annotation_DF.head()

,isolate_ID,lineage_call
0,SAMEA3558733,4.3.i4.2
1,SAMN03648641,4.4.1.1
2,SAMN03647419,3.1.1.i1
3,SAMEA3671418,4.3.i2
4,SAMN07659096,1.1.3


In [32]:
np.shape(SNP_isolate_annotation_DF)

(31428, 2)

In [33]:
SNP_annotation_DF.head()

,index,pos,ref,alt
0,0,48,C,[T]
1,1,64,G,[C]
2,2,67,G,"[A, T]"
3,3,69,C,[T]
4,4,71,C,[T]


In [34]:
np.shape(SNP_annotation_DF)

(782565, 4)

In [35]:
SNP_genotypes_array

array([[1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [1, 9, 9, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1],
       [1, 9, 1, ..., 1, 1, 1]], dtype=int8)

In [36]:
np.shape(SNP_genotypes_array)

(782565, 31428)

### additional 12 eis C-14T mutants with AG MICs

In [37]:
#load isolate annotation file (columns of Genotype Matrix)
isolate_annotation_DF_extra_strains = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/eis_promoter_mutant_strains_from_Milan/Genotypes/Genotypes_Filtered/genotypes_isolate_annotation.pkl')
isolate_annotation_DF_extra_strains = isolate_annotation_DF_extra_strains.loc[:, ['isolate_ID','lineage_call']]
isolate_annotation_DF_extra_strains = convert_lineage_calls(isolate_annotation_DF_extra_strains)

#load Genotypes Matrix
genotypes_array_extra_strains =  np.load('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/eis_promoter_mutant_strains_from_Milan/Genotypes/Genotypes_Filtered/genotypes_matrix.npy')

In [38]:
isolate_annotation_DF_extra_strains.head()

,isolate_ID,lineage_call
0,168-19,2.2.1.1.1
1,622-19,2.2.1.1.1.i3
2,655-19,4.3.i3.1
3,IT1070,2.2.1.1.1.i3
4,IT123,2.2.1.1.1


In [39]:
genotypes_array_extra_strains

array([[1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int8)

In [40]:
np.shape(genotypes_array_extra_strains)

(782565, 12)

### Merge *isolate annotation files* and *genotypes matrices* together

In [41]:
# merge isolate annotation dataframes
SNP_isolate_annotation_DF = SNP_isolate_annotation_DF.append(isolate_annotation_DF_extra_strains)
SNP_isolate_annotation_DF.reset_index(inplace = True, drop = True)

In [42]:
np.shape(SNP_isolate_annotation_DF)

(31440, 2)

In [43]:
# merge genotypes matrices
SNP_genotypes_array = np.hstack((SNP_genotypes_array, genotypes_array_extra_strains))

In [44]:
np.shape(SNP_genotypes_array)

(782565, 31440)

## [2.2] Subset to targeted chomosomal region

In [46]:
# keep only SNPs that occur within chromosomal region
SNPs_in_targeted_region = list(SNP_annotation_DF[((SNP_annotation_DF.pos >= H37Rv_region_start_A) & (SNP_annotation_DF.pos <= H37Rv_region_end_A)) | ((SNP_annotation_DF.pos >= H37Rv_region_start_B) & (SNP_annotation_DF.pos <= H37Rv_region_end_B))].pos)

# construct boolean filter
SNPs_to_keep_filter = [SNP_i in SNPs_in_targeted_region for SNP_i in SNP_annotation_DF.pos]

# filter Genotypes Matrix
SNP_genotypes_array = SNP_genotypes_array[SNPs_to_keep_filter , :]

# filter SNP annotation file
SNP_annotation_DF = SNP_annotation_DF[SNPs_to_keep_filter]
SNP_annotation_DF.reset_index(drop = True , inplace = True) #re-index new filtered SNP annotation DF (so new index matches indexing of genotypes matrix rows)
SNP_annotation_DF.loc[:,'array_index'] = SNP_annotation_DF.index

/home/rv76/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/rv76/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [47]:
SNP_annotation_DF.head()

,index,pos,ref,alt,array_index
0,489761,2714130,C,[G],0
1,489762,2714139,G,[A],1
2,489763,2714140,G,[T],2
3,489764,2714145,G,[A],3
4,489765,2714147,C,[A],4


In [48]:
np.shape(SNP_annotation_DF)

(343, 5)

#### Re-format SNP annotation DataFrame so that each alternate allele is its own row

In [50]:
SNP_annotation_DF_mod = pd.DataFrame(columns = SNP_annotation_DF.columns)
new_SNP_index = 0
for SNP_i_row in SNP_annotation_DF.index:
    
    SNP_i_info = SNP_annotation_DF.loc[SNP_i_row, :]
    
    #iterate through alternate alleles, store a row for each one
    for SNP_i_alt_allele in SNP_i_info.alt:
        
        SNP_annotation_DF_mod.loc[new_SNP_index, :] = list(SNP_i_info.loc[['index','pos','ref']]) + [SNP_i_alt_allele] + list(SNP_i_info.loc[['array_index']])
        new_SNP_index += 1

SNP_annotation_DF = SNP_annotation_DF_mod
#create new column to merge func annotation on
SNP_annotation_DF.loc[:, 'key'] = [str(SNP_i_pos) + '_' + SNP_i_alt for SNP_i_pos, SNP_i_alt in zip(SNP_annotation_DF.pos, SNP_annotation_DF.alt)]

In [51]:
SNP_annotation_DF.head()

,index,pos,ref,alt,array_index,key
0,489761,2714130,C,G,0,2714130_G
1,489762,2714139,G,A,1,2714139_A
2,489763,2714140,G,T,2,2714140_T
3,489764,2714145,G,A,3,2714145_A
4,489765,2714147,C,A,4,2714147_A


In [52]:
np.shape(SNP_annotation_DF)

(365, 6)

#### Merge functional annotation data

In [53]:
SNP_func_annotation_DF = pd.read_pickle('/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_functional_annotation.pkl')
SNP_func_annotation_DF.loc[:, 'key'] = SNP_func_annotation_DF.index

In [54]:
SNP_func_annotation_DF.head()

,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,SNP_type,AA_change,key
48_T,48,C,T,Essential,dnaA,Rv0001,48,S,V16V,48_T
64_C,64,G,C,Essential,dnaA,Rv0001,64,N,G22R,64_C
67_A,67,G,A,Essential,dnaA,Rv0001,67,N,D23N,67_A
67_T,67,G,T,Essential,dnaA,Rv0001,67,N,D23Y,67_T
69_T,69,C,T,Essential,dnaA,Rv0001,69,S,D23D,69_T


In [55]:
SNP_annotation_DF = SNP_annotation_DF.merge(SNP_func_annotation_DF.loc[: , ['gene_category','gene_name','gene_id','gene_pos','SNP_type','AA_change','key']], how = 'left', on = 'key')
SNP_annotation_DF = SNP_annotation_DF.loc[:, ['key','pos','ref','alt','gene_category','gene_name','gene_id','gene_pos','SNP_type','AA_change','array_index']] #re-arrange columns
SNP_annotation_DF.rename(columns={'SNP_type':'mut_type', 'AA_change':'codon_pos'}, inplace = True) #change column names

In [56]:
SNP_annotation_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,mut_type,codon_pos,array_index
0,2714130_G,2714130,C,G,Non-Essential,eis,Rv2416c,1203,N,E401D,0
1,2714139_A,2714139,G,A,Non-Essential,eis,Rv2416c,1194,S,T398T,1
2,2714140_T,2714140,G,T,Non-Essential,eis,Rv2416c,1193,N,T398N,2
3,2714145_A,2714145,G,A,Non-Essential,eis,Rv2416c,1188,S,V396V,3
4,2714147_A,2714147,C,A,Non-Essential,eis,Rv2416c,1186,N,V396F,4


In [57]:
np.shape(SNP_annotation_DF)

(365, 11)

In [58]:
SNP_genotypes_array

array([[1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int8)

In [59]:
np.shape(SNP_genotypes_array)

(343, 31440)

# [3] Pull info for isolates with mutant alleles

## [3.1] Get genotypes for SNPs of interest

### *Function* to get the genotypes for a specifc SNP and get attributes for isolates that support SNP

In [61]:
def get_genotypes_for_SNP(SNP_i_pos, SNP_i_alt_allele):
    
    #get the number code that corresponds to the alt allele, 9 indicated bad quality call
    base_code_dict = {'A':0, 'C':1, 'G':2, 'T':3}
    SNP_i_alt_allele_code = base_code_dict[SNP_i_alt_allele]
    SNP_i_genotypes = SNP_genotypes_array[SNP_annotation_DF[SNP_annotation_DF.pos == SNP_i_pos].array_index.values[0] , :] 

    #count the number of isolates that support this SNP call
    num_isolates_SNP_i = np.sum(SNP_i_genotypes == SNP_i_alt_allele_code)

    #create a boolean filter and extract information for the isolates that support this call
    SNP_i_isolate_filter = SNP_i_genotypes == SNP_i_alt_allele_code
    isolate_annotation_SNP_i = SNP_isolate_annotation_DF[SNP_i_isolate_filter]

    #find number of sub-lineages that have SNP support in at least 1 isolate
    num_sublineages_with_SNP_i = len(set(list(isolate_annotation_SNP_i.lineage_call.values)))
    
    #get list of sublineages w/ at least 1 isolates that supports SNP call
    sublineages_with_SNP_count_dict = Counter(isolate_annotation_SNP_i.lineage_call.values)
    sublineages_with_SNP_list = []
    for sublineage_i in sublineages_with_SNP_count_dict.keys():

        sublineage_i_with_SNP = sublineage_i + '({0})'.format(str(sublineages_with_SNP_count_dict[sublineage_i]))
        sublineages_with_SNP_list.append(sublineage_i_with_SNP)

    sublineages_with_SNP = ' - '.join(sublineages_with_SNP_list)

    return [num_isolates_SNP_i , num_sublineages_with_SNP_i , sublineages_with_SNP]

In [62]:
SNP_annotation_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,mut_type,codon_pos,array_index
0,2714130_G,2714130,C,G,Non-Essential,eis,Rv2416c,1203,N,E401D,0
1,2714139_A,2714139,G,A,Non-Essential,eis,Rv2416c,1194,S,T398T,1
2,2714140_T,2714140,G,T,Non-Essential,eis,Rv2416c,1193,N,T398N,2
3,2714145_A,2714145,G,A,Non-Essential,eis,Rv2416c,1188,S,V396V,3
4,2714147_A,2714147,C,A,Non-Essential,eis,Rv2416c,1186,N,V396F,4


In [63]:
np.shape(SNP_annotation_DF)

(365, 11)

In [64]:
num_isolates_with_SNP = []
num_sublineages_with_SNP = []
sublineages_with_SNP = []

for SNP_i_pos, SNP_i_alt_allele in zip(SNP_annotation_DF.pos, SNP_annotation_DF.alt):
    
    num_isolates_with_SNP_i , num_sublineages_with_SNP_i , sublineages_with_SNP_i = get_genotypes_for_SNP(SNP_i_pos, SNP_i_alt_allele)
    
    num_isolates_with_SNP.append(num_isolates_with_SNP_i)
    num_sublineages_with_SNP.append(num_sublineages_with_SNP_i)
    sublineages_with_SNP.append(sublineages_with_SNP_i)
    
SNP_annotation_DF.loc[: , 'num_isolates'] = num_isolates_with_SNP
SNP_annotation_DF.loc[: , 'num_sublineages'] = num_sublineages_with_SNP
SNP_annotation_DF.loc[: , 'sublineages'] = sublineages_with_SNP

#drop SNPs present in 0 isolates
SNP_annotation_DF = SNP_annotation_DF[SNP_annotation_DF.num_isolates > 0]
SNP_annotation_DF.reset_index(inplace = True, drop = True)

In [65]:
SNP_annotation_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,mut_type,codon_pos,array_index,num_isolates,num_sublineages,sublineages
0,2714130_G,2714130,C,G,Non-Essential,eis,Rv2416c,1203,N,E401D,0,27,2,3.1.1(25) - 3.1(2)
1,2714139_A,2714139,G,A,Non-Essential,eis,Rv2416c,1194,S,T398T,1,1,1,4.1.i1.1.1.1(1)
2,2714140_T,2714140,G,T,Non-Essential,eis,Rv2416c,1193,N,T398N,2,2,1,1.1.1.1(2)
3,2714145_A,2714145,G,A,Non-Essential,eis,Rv2416c,1188,S,V396V,3,2,2,4.5(1) - 4.13(1)
4,2714147_A,2714147,C,A,Non-Essential,eis,Rv2416c,1186,N,V396F,4,1,1,1.1.3(1)


In [66]:
np.shape(SNP_annotation_DF)

(364, 14)

## [3.2] Get genotypes for indels of interest

### *Function* to get the genotypes for a specifc indel and get attributes for isolates that support indel

In [67]:
def get_genotypes_for_indel(indel_i_key):
    
    #0 supports Ref, 1 supports Alt, 9 indicated bad quality call
    indel_i_genotypes = INDEL_genotypes_array[INDEL_annotation_DF[INDEL_annotation_DF.key == indel_i_key].index[0] , :] 

    #count the number of isolates that support this indel call
    num_isolates_indel_i = np.sum(indel_i_genotypes == 1)

    #create a boolean filter and extract information for the isolates taht support this call
    indel_i_isolate_filter = indel_i_genotypes == 1
    isolate_annotation_indel_i = INDEL_isolate_annotation_DF[indel_i_isolate_filter]

    #find number of sub-lineages that have indel support in at least 1 isolate
    num_sublineages_with_indel_i = len(set(list(isolate_annotation_indel_i.lineage_call.values)))
    
    #get list of sublineages w/ at least 1 isolates that supports indel call
    sublineage_with_indel_count_dict = Counter(isolate_annotation_indel_i.lineage_call.values)
    sublineages_with_indel_list = []
    for sublineage_i in sublineage_with_indel_count_dict.keys():

        sublineage_i_with_indel = sublineage_i + '({0})'.format(str(sublineage_with_indel_count_dict[sublineage_i]))
        sublineages_with_indel_list.append(sublineage_i_with_indel)

    sublineages_with_indel = ' - '.join(sublineages_with_indel_list)
    
    return [num_isolates_indel_i , num_sublineages_with_indel_i , sublineages_with_indel]

In [68]:
INDEL_annotation_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,ins_del,INDEL_type,codon_pos
0,A_2714157_ACTG,2714157,A,ACTG,Non-Essential,eis,Rv2416c,1176,ins,inframe,392
1,T_2714272_TC,2714272,T,TC,Non-Essential,eis,Rv2416c,1061,ins,frameshift,354
2,C_2714303_CG,2714303,C,CG,Non-Essential,eis,Rv2416c,1030,ins,frameshift,344
3,G_2714334_GA,2714334,G,GA,Non-Essential,eis,Rv2416c,999,ins,frameshift,333
4,ACTCG_2714384_A,2714384,ACTCG,A,Non-Essential,eis,Rv2416c,949,del,frameshift,317


In [69]:
np.shape(INDEL_annotation_DF)

(40, 11)

In [70]:
num_isolates_with_indel = []
num_sublineages_with_indel = []
sublineages_with_indel = []

for indel_i_key in INDEL_annotation_DF.key:
    
    num_isolates_with_indel_i , num_sublineages_with_indel_i , sublineages_with_indel_i = get_genotypes_for_indel(indel_i_key)
    
    num_isolates_with_indel.append(num_isolates_with_indel_i)
    num_sublineages_with_indel.append(num_sublineages_with_indel_i)
    sublineages_with_indel.append(sublineages_with_indel_i)
    
INDEL_annotation_DF.loc[: , 'num_isolates'] = num_isolates_with_indel
INDEL_annotation_DF.loc[: , 'num_sublineages'] = num_sublineages_with_indel
INDEL_annotation_DF.loc[: , 'sublineages'] = sublineages_with_indel

#drop indels present in 0 isolates
INDEL_annotation_DF = INDEL_annotation_DF[INDEL_annotation_DF.num_isolates > 0]

In [71]:
INDEL_annotation_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,ins_del,INDEL_type,codon_pos,num_isolates,num_sublineages,sublineages
2,C_2714303_CG,2714303,C,CG,Non-Essential,eis,Rv2416c,1030,ins,frameshift,344,3,1,4.1.i1.1.1.1(3)
3,G_2714334_GA,2714334,G,GA,Non-Essential,eis,Rv2416c,999,ins,frameshift,333,1,1,3.1.1(1)
4,ACTCG_2714384_A,2714384,ACTCG,A,Non-Essential,eis,Rv2416c,949,del,frameshift,317,4,1,4.4.1.1(4)
5,GT_2714433_G,2714433,GT,G,Non-Essential,eis,Rv2416c,900,del,frameshift,300,2,1,4.1.i1.2.1(2)
6,GGT_2714526_G,2714526,GGT,G,Non-Essential,eis,Rv2416c,807,del,frameshift,269,112,1,2.2.1.1.1(112)


In [72]:
np.shape(INDEL_annotation_DF)

(36, 14)

# [4] Check co-occurrence between *regulator* SNPs/INDELs & *regulon* SNPs/INDELs

### SNPs in regulator

In [73]:
regulator_SNPs_DF = SNP_annotation_DF[SNP_annotation_DF.gene_id == regulator_id]

In [74]:
regulator_SNPs_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,mut_type,codon_pos,array_index,num_isolates,num_sublineages,sublineages
263,3568686_C,3568686,T,C,None,None,Rv3197A_Rv3198c,None,I,None,249,1,1,1.2.2.1(1)
264,3568686_G,3568686,T,G,None,None,Rv3197A_Rv3198c,None,I,None,249,1,1,2.2.1.1.1(1)
265,3568692_T,3568692,C,T,None,None,Rv3197A_Rv3198c,None,I,None,250,1,1,1.2.2.1(1)
266,3568701_A,3568701,G,A,None,None,Rv3197A_Rv3198c,None,I,None,251,1,1,4.10.i1(1)
267,3568702_T,3568702,G,T,None,None,Rv3197A_Rv3198c,None,I,None,252,1,1,3.1.1(1)


In [75]:
np.shape(regulator_SNPs_DF)

(101, 14)

### INDELs in regulator

In [76]:
regulator_INDELs_DF = INDEL_annotation_DF[INDEL_annotation_DF.gene_id == regulator_id]

In [77]:
regulator_INDELs_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,ins_del,INDEL_type,codon_pos,num_isolates,num_sublineages,sublineages
25,C_3568730_CG,3568730,C,CG,None,None,Rv3197A_Rv3198c,None,ins,frameshift,None,1,1,4.10.i1(1)
26,CA_3568735_C,3568735,CA,C,None,None,Rv3197A_Rv3198c,None,del,frameshift,None,1,1,3.1.1(1)
27,T_3568795_TC,3568795,T,TC,None,None,Rv3197A_Rv3198c,None,ins,frameshift,None,1,1,4.1.i1.1.1.1(1)
28,AC_3568854_A,3568854,AC,A,None,None,Rv3197A_Rv3198c,None,del,frameshift,None,1,1,2.2.1.1.1(1)
29,C_3568881_CGG,3568881,C,CGG,None,None,Rv3197A_Rv3198c,None,ins,frameshift,None,1,1,4.3.i3.1(1)


In [78]:
np.shape(regulator_INDELs_DF)

(15, 14)

Drop **INDEL_type** column and rename **ins_del** column to fit with SNPs DataFrames

In [79]:
regulator_INDELs_DF.drop(['INDEL_type'], axis=1, inplace = True)
regulator_INDELs_DF.rename(columns={'ins_del':'mut_type'}, inplace = True) #change column names

/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/rv76/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


### SNPs in regulon

We're going to include SNPs in a gene that either 

- cause a **premature STOP codon** (nonsense mutation)
- **synonymous mutations at the first codon that abolish the start codon** (i.e. eis, whiB7 and mmpR which start with a **valine** as this is not possible with ahpC, mmpS5 or mmpL5, which start with a **methionine**)
- **non-synonymous mutations at the first codon that abolish the start codon** (i.e. that change the start to an AA other than **valine** and **methionine**)

In [80]:
regulon_SNPs_DF = SNP_annotation_DF[SNP_annotation_DF.gene_id == regulon_id]

In [81]:
regulon_SNPs_DF[[(AA_change[-1]=='*') | ((len(AA_change)==3) and (AA_change[1]=='1')) for AA_change in regulon_SNPs_DF.codon_pos]]

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,mut_type,codon_pos,array_index,num_isolates,num_sublineages,sublineages
59,2714449_T,2714449,C,T,Non-Essential,eis,Rv2416c,884,N,W295*,57,1,1,3.1.1(1)
82,2714543_A,2714543,G,A,Non-Essential,eis,Rv2416c,790,N,R264*,79,1,1,3.1.1(1)
103,2714664_T,2714664,G,T,Non-Essential,eis,Rv2416c,669,N,Y223*,99,2,1,3.1.1(2)
123,2714753_A,2714753,G,A,Non-Essential,eis,Rv2416c,580,N,Q194*,118,10,2,4.10.i1(1) - 1.2.2.1(9)
132,2714787_T,2714787,C,T,Non-Essential,eis,Rv2416c,546,N,W182*,126,15,1,4.10.i1(15)
244,2715294_T,2715294,C,T,Non-Essential,eis,Rv2416c,39,N,W13*,232,1,1,2.2.1.1.1(1)
245,2715295_T,2715295,C,T,Non-Essential,eis,Rv2416c,38,N,W13*,233,1,1,2.2.1.1.1(1)
249,2715305_A,2715305,C,A,Non-Essential,eis,Rv2416c,28,N,E10*,237,6,2,1.1.3(5) - 1.1.1.1(1)
262,2715330_T,2715330,C,T,Non-Essential,eis,Rv2416c,3,S,V1V,248,6,2,2.2.1.1.1.i3(5) - 1.1.1.1(1)


In [82]:
regulon_SNPs_DF = regulon_SNPs_DF[[(AA_change[-1]=='*') | ((len(AA_change)==3) and (AA_change[1]=='1')) for AA_change in regulon_SNPs_DF.codon_pos]]

Save SNPs that occur in the first codon seperately

In [75]:
# regulon_SNPs_DF[[((len(AA_change)==3) and (AA_change[1]=='1')) for AA_change in regulon_SNPs_DF.codon_pos]].to_excel('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/CSV files/START_codon_SNVs/whiB7_START_codon_SNVs.xlsx')

### INDELs in regulon

#### keep only *frameshift* indels for gene body

In [83]:
INDEL_annotation_DF[(INDEL_annotation_DF.gene_id == regulon_id) & (INDEL_annotation_DF.INDEL_type == 'frameshift')].head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,ins_del,INDEL_type,codon_pos,num_isolates,num_sublineages,sublineages
2,C_2714303_CG,2714303,C,CG,Non-Essential,eis,Rv2416c,1030,ins,frameshift,344,3,1,4.1.i1.1.1.1(3)
3,G_2714334_GA,2714334,G,GA,Non-Essential,eis,Rv2416c,999,ins,frameshift,333,1,1,3.1.1(1)
4,ACTCG_2714384_A,2714384,ACTCG,A,Non-Essential,eis,Rv2416c,949,del,frameshift,317,4,1,4.4.1.1(4)
5,GT_2714433_G,2714433,GT,G,Non-Essential,eis,Rv2416c,900,del,frameshift,300,2,1,4.1.i1.2.1(2)
6,GGT_2714526_G,2714526,GGT,G,Non-Essential,eis,Rv2416c,807,del,frameshift,269,112,1,2.2.1.1.1(112)


In [84]:
regulon_INDELs_DF = INDEL_annotation_DF[(INDEL_annotation_DF.gene_id == regulon_id) & (INDEL_annotation_DF.INDEL_type == 'frameshift')]

In [85]:
regulon_INDELs_DF.head()

,key,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,ins_del,INDEL_type,codon_pos,num_isolates,num_sublineages,sublineages
2,C_2714303_CG,2714303,C,CG,Non-Essential,eis,Rv2416c,1030,ins,frameshift,344,3,1,4.1.i1.1.1.1(3)
3,G_2714334_GA,2714334,G,GA,Non-Essential,eis,Rv2416c,999,ins,frameshift,333,1,1,3.1.1(1)
4,ACTCG_2714384_A,2714384,ACTCG,A,Non-Essential,eis,Rv2416c,949,del,frameshift,317,4,1,4.4.1.1(4)
5,GT_2714433_G,2714433,GT,G,Non-Essential,eis,Rv2416c,900,del,frameshift,300,2,1,4.1.i1.2.1(2)
6,GGT_2714526_G,2714526,GGT,G,Non-Essential,eis,Rv2416c,807,del,frameshift,269,112,1,2.2.1.1.1(112)


In [86]:
np.shape(regulon_INDELs_DF)

(18, 14)

Drop **INDEL_type** column (since all indels are frameshifts) and rename **ins_del** column to fit with SNPs DataFrames

In [87]:
regulon_INDELs_DF.drop(['INDEL_type'], axis=1, inplace = True)
regulon_INDELs_DF.rename(columns={'ins_del':'mut_type'}, inplace = True) #change column names

/home/rv76/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Save DataFrame for *regulator* & *regulon* mutations

In [88]:
regulator_SNPs_DF_for_export = regulator_SNPs_DF.drop(['array_index'], axis=1)
regulon_SNPs_DF_for_export = regulon_SNPs_DF.drop(['array_index'], axis=1)
regulator_regulon_mutations_df = regulator_SNPs_DF_for_export.append(regulon_SNPs_DF_for_export.append(regulator_INDELs_DF.append(regulon_INDELs_DF)))

regulator_regulon_mutations_df.drop(['key'], axis=1, inplace = True) #drop col
regulator_regulon_mutations_df.sort_values(by = 'pos', inplace = True) #sort by reference position
regulator_regulon_mutations_df.reset_index(inplace = True, drop = True) #reset index

In [89]:
np.shape(regulator_regulon_mutations_df)

(143, 12)

In [90]:
regulator_regulon_mutations_df

,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,mut_type,codon_pos,num_isolates,num_sublineages,sublineages
0,2714303,C,CG,Non-Essential,eis,Rv2416c,1030,ins,344,3,1,4.1.i1.1.1.1(3)
1,2714334,G,GA,Non-Essential,eis,Rv2416c,999,ins,333,1,1,3.1.1(1)
2,2714384,ACTCG,A,Non-Essential,eis,Rv2416c,949,del,317,4,1,4.4.1.1(4)
3,2714433,GT,G,Non-Essential,eis,Rv2416c,900,del,300,2,1,4.1.i1.2.1(2)
4,2714449,C,T,Non-Essential,eis,Rv2416c,884,N,W295*,1,1,3.1.1(1)
5,2714526,GGT,G,Non-Essential,eis,Rv2416c,807,del,269,112,1,2.2.1.1.1(112)
6,2714543,G,A,Non-Essential,eis,Rv2416c,790,N,R264*,1,1,3.1.1(1)
7,2714557,AGGCCAATCAG,A,Non-Essential,eis,Rv2416c,776,del,259,1,1,2.2.1.1.1(1)
8,2714664,G,T,Non-Essential,eis,Rv2416c,669,N,Y223*,2,1,3.1.1(2)
9,2714740,TC,T,Non-Essential,eis,Rv2416c,593,del,198,2,1,1.1.2(2)


In [85]:
# regulator_regulon_mutations_df.to_excel('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/CSV files/mutations_detected_within_isolates/promoter_whiB7_gene_body_mutations_in_31440_isolates.xlsx')

#### Save DataFrame for *regulator* & *regulon* mutations for *Table 1*

**REGULATOR** Subset to the 4 most commonly occurring mutations in the regulator regions

In [91]:
regulator_mutations_df_subset = regulator_regulon_mutations_df[regulator_regulon_mutations_df.gene_id == regulator_id].sort_values(by = 'num_isolates', ascending = False).head(n=4)

**REGULON** Subset to mutations that occured in at least 2 Mtb sub-lineages in the regulon regions

In [92]:
regulon_mutations_df_subset = regulator_regulon_mutations_df[regulator_regulon_mutations_df.gene_id == regulon_id]
regulon_mutations_df_subset = regulon_mutations_df_subset[regulon_mutations_df_subset.num_sublineages >= 2]

Export as excel file

In [93]:
regulator_regulon_subset_mutations_df = regulator_mutations_df_subset.append(regulon_mutations_df_subset)
regulator_regulon_subset_mutations_df.sort_values(by = 'pos', ascending = True, inplace = True)
regulator_regulon_subset_mutations_df.reset_index(inplace = True, drop = True)

In [94]:
regulator_regulon_subset_mutations_df

,pos,ref,alt,gene_category,gene_name,gene_id,gene_pos,mut_type,codon_pos,num_isolates,num_sublineages,sublineages
0,2714753,G,A,Non-Essential,eis,Rv2416c,580,N,Q194*,10,2,4.10.i1(1) - 1.2.2.1(9)
1,2715287,T,TC,Non-Essential,eis,Rv2416c,46,ins,16,3,2,4.4.1.1(1) - 4.3.i2(2)
2,2715305,C,A,Non-Essential,eis,Rv2416c,28,N,E10*,6,2,1.1.3(5) - 1.1.1.1(1)
3,2715330,C,T,Non-Essential,eis,Rv2416c,3,S,V1V,6,2,2.2.1.1.1.i3(5) - 1.1.1.1(1)
4,3568779,A,G,None,None,Rv3197A_Rv3198c,None,I,None,256,2,4.4.1.1(244) - 4.2.1.2.2.1(12)
5,3568857,G,A,None,None,Rv3197A_Rv3198c,None,I,None,73,2,3.1.1(72) - 3.1(1)
6,3568921,C,G,None,None,Rv3197A_Rv3198c,None,I,None,117,2,4.1.i1.1.2(110) - 4.1.i1.1(7)
7,3569029,T,C,None,None,Rv3197A_Rv3198c,None,I,None,249,3,4.2.1.1.1.2.2.1.1.1(5) - 4.6.1.1.1.1.2(173) - ...


In [90]:
# regulator_regulon_subset_mutations_df.to_excel('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/CSV files/mutations_detected_within_isolates/promoter_whiB7_gene_body_mutations_in_31440_isolates_for_Table_1.xlsx')

### Construct DataFrame (which displays each pair of co-occuring mutations as a seperate row with info for each pair of mutations)

In [95]:
variant_mutation_pair_df = pd.DataFrame(columns=['A_type', 'A_pos', 'A_ref', 'A_alt', 'A_mut_type', 'A_codon_pos', 'A_gene_id', 'A_gene_name', 'A_gene_pos' ,'A_num_isolates', 'B_type', 'B_pos', 'B_ref', 'B_alt', 'B_mut_type', 'B_codon_pos', 'B_gene_id', 'B_gene_name', 'B_gene_pos', 'B_num_isolates', 'num_isolates_co_occur', 'isolate_IDs', 'sublineages'])

### Get the ordering of the INDELs genotypes columns in terms of the SNPs genotypes columns

In [96]:
SNP_isolate_annotation_DF['isolate_order'] = SNP_isolate_annotation_DF.index
SNP_isolate_annotation_DF.set_index('isolate_ID', drop = True, inplace = True)
INDEL_isolate_ordering_for_SNP_genotypes_array = SNP_isolate_annotation_DF.loc[INDEL_isolate_annotation_DF.isolate_ID, :].isolate_order.values

In [97]:
INDEL_isolate_ordering_for_SNP_genotypes_array

array([  188,  2733, 24409, ..., 31437, 31438, 31439])

### Fill in dataframe by quering the genotypes matrices

In [98]:
#SNPs or INDELs in regulator x SNPs or INDELs in regulon
regulator_mut_keys = list(regulator_SNPs_DF.key) + list(regulator_INDELs_DF.key)
regulon_mut_keys = list(regulon_SNPs_DF.key) + list(regulon_INDELs_DF.key)

In [99]:
#index for DataFrame
variant_mutation_pair_df_index = 0

for variant_i in regulator_mut_keys: #iterate through regulator SNPs & INDELs
    
    if variant_i.count('_') == 1: # if variant in regulator is a SNP

        SNP_i_pos = int(variant_i.split('_')[0])
        SNP_i_alt_allele = variant_i.split('_')[1]

        #get the number code that corresponds to the alt allele, 9 indicated bad quality call
        base_code_dict = {'A':0, 'C':1, 'G':2, 'T':3}
        SNP_i_alt_allele_code = base_code_dict[SNP_i_alt_allele]

        #genotypes for each SNP site
        genotypes_for_SNP_i = SNP_genotypes_array[SNP_annotation_DF[SNP_annotation_DF.pos == SNP_i_pos].array_index.values[0] , :]

        #re-order isolates according to the order of the columns in the INDEL genotypes matrix
        genotypes_for_SNP_i = genotypes_for_SNP_i[INDEL_isolate_ordering_for_SNP_genotypes_array]

        for variant_j in regulon_mut_keys: #iterate through mutations in regulon

            if variant_j.count('_') == 1: #if variant in regulon is a SNP

                SNP_j_pos = int(variant_j.split('_')[0])
                SNP_j_alt_allele = variant_j.split('_')[1]

                #get the number code that corresponds to the alt allele, 9 indicated bad quality call
                SNP_j_alt_allele_code = base_code_dict[SNP_j_alt_allele]

                #genotypes for each SNP site
                genotypes_for_SNP_j = SNP_genotypes_array[SNP_annotation_DF[SNP_annotation_DF.pos == SNP_j_pos].array_index.values[0] , :]

                #re-order isolates according to the order of the columns in the INDEL genotypes matrix
                genotypes_for_SNP_j = genotypes_for_SNP_j[INDEL_isolate_ordering_for_SNP_genotypes_array]

                #boolean filter of isolates with both variants
                isolates_with_both_variants_filter = [(SNP_i_called and SNP_j_called) for SNP_i_called , SNP_j_called in zip(list(genotypes_for_SNP_i == SNP_i_alt_allele_code) , list(genotypes_for_SNP_j == SNP_j_alt_allele_code))]

                #number of isolates that have both variants
                num_isolates_with_both_variants = np.sum(isolates_with_both_variants_filter)

                #check to see if the number of isolates carrying both variants was >= 1
                if num_isolates_with_both_variants >= 1:

                    # get info for mutation in regulator
                    regulator_SNP_info = list(regulator_SNPs_DF[(regulator_SNPs_DF.pos == SNP_i_pos) & (regulator_SNPs_DF.alt == SNP_i_alt_allele)].loc[:, ['pos','ref','alt','mut_type','codon_pos','gene_id','gene_name','gene_pos','num_isolates']].values[0])

                    # get info for mutation in regulon
                    regulon_SNP_info = list(regulon_SNPs_DF[(regulon_SNPs_DF.pos == SNP_j_pos) & (regulon_SNPs_DF.alt == SNP_j_alt_allele)].loc[:, ['pos','ref','alt','mut_type','codon_pos','gene_id','gene_name','gene_pos','num_isolates']].values[0])

                    strain_names = [' - '.join(list(INDEL_isolate_annotation_DF[isolates_with_both_variants_filter].isolate_ID))]

                    # sublineage calls
                    strain_lineage_calls_count_dict = Counter(list(INDEL_isolate_annotation_DF[isolates_with_both_variants_filter].lineage_call))
                    strain_lineage_calls_list = []
                    for sublineage_i in strain_lineage_calls_count_dict.keys():
                        sublineage_i_with_mutations = sublineage_i + '({0})'.format(str(strain_lineage_calls_count_dict[sublineage_i]))
                        strain_lineage_calls_list.append(sublineage_i_with_mutations)
                    strain_lineage_calls = ' - '.join(strain_lineage_calls_list)

                    co_occurring_SNP_i_and_SNP_or_indel_j_info = ['SNP'] + regulator_SNP_info + ['SNP'] + regulon_SNP_info + [num_isolates_with_both_variants] + strain_names + [strain_lineage_calls]
                    variant_mutation_pair_df.loc[variant_mutation_pair_df_index, :] = co_occurring_SNP_i_and_SNP_or_indel_j_info
                    variant_mutation_pair_df_index += 1

            if variant_j.count('_') == 2: #if variant in regulon is an INDEL

                #genotypes for each INDEL
                genotypes_for_indel_j = INDEL_genotypes_array[INDEL_annotation_DF[INDEL_annotation_DF.key == variant_j].index[0] , :]

                #boolean filter of isolates with both variants
                isolates_with_both_variants_filter = [(SNP_i_called and indel_j_called) for SNP_i_called , indel_j_called in zip(list(genotypes_for_SNP_i == SNP_i_alt_allele_code) , list(genotypes_for_indel_j == 1))]

                #number of isolates that have both variants
                num_isolates_with_both_variants = np.sum(isolates_with_both_variants_filter)

                #check to see if the number of isolates carrying both variants was >= 1
                if num_isolates_with_both_variants >= 1:

                    # get info for mutation in regulator
                    regulator_SNP_info = list(regulator_SNPs_DF[(regulator_SNPs_DF.pos == SNP_i_pos) & (regulator_SNPs_DF.alt == SNP_i_alt_allele)].loc[:, ['pos','ref','alt','mut_type','codon_pos','gene_id','gene_name','gene_pos','num_isolates']].values[0])

                    # get info for mutation in regulon
                    regulon_INDEL_info = list(regulon_INDELs_DF[regulon_INDELs_DF.key == variant_j].loc[:, ['pos','ref','alt','mut_type','codon_pos','gene_id','gene_name','gene_pos','num_isolates']].values[0])

                    strain_names = [' - '.join(list(INDEL_isolate_annotation_DF[isolates_with_both_variants_filter].isolate_ID))]

                    # sublineage calls
                    strain_lineage_calls_count_dict = Counter(list(INDEL_isolate_annotation_DF[isolates_with_both_variants_filter].lineage_call))
                    strain_lineage_calls_list = []
                    for sublineage_i in strain_lineage_calls_count_dict.keys():
                        sublineage_i_with_mutations = sublineage_i + '({0})'.format(str(strain_lineage_calls_count_dict[sublineage_i]))
                        strain_lineage_calls_list.append(sublineage_i_with_mutations)
                    strain_lineage_calls = ' - '.join(strain_lineage_calls_list)

                    co_occurring_SNP_i_and_SNP_or_indel_j_info = ['SNP'] + regulator_SNP_info + ['INDEL'] + regulon_INDEL_info + [num_isolates_with_both_variants] + strain_names + [strain_lineage_calls]
                    variant_mutation_pair_df.loc[variant_mutation_pair_df_index, :] = co_occurring_SNP_i_and_SNP_or_indel_j_info
                    variant_mutation_pair_df_index += 1
    
    elif variant_i.count('_') == 2: # if variant in regulator is an INDEL
        
        #genotypes for each INDEL
        genotypes_for_indel_i = INDEL_genotypes_array[INDEL_annotation_DF[INDEL_annotation_DF.key == variant_i].index[0] , :]

        for variant_j in regulon_mut_keys: #iterate through mutations in regulon

            if variant_j.count('_') == 1: #if variant in regulon is a SNP

                SNP_j_pos = int(variant_j.split('_')[0])
                SNP_j_alt_allele = variant_j.split('_')[1]

                #get the number code that corresponds to the alt allele, 9 indicated bad quality call
                SNP_j_alt_allele_code = base_code_dict[SNP_j_alt_allele]

                #genotypes for each SNP site
                genotypes_for_SNP_j = SNP_genotypes_array[SNP_annotation_DF[SNP_annotation_DF.pos == SNP_j_pos].array_index.values[0] , :]

                #re-order isolates according to the order of the columns in the INDEL genotypes matrix
                genotypes_for_SNP_j = genotypes_for_SNP_j[INDEL_isolate_ordering_for_SNP_genotypes_array]

                #boolean filter of isolates with both variants
                isolates_with_both_variants_filter = [(indel_i_called and SNP_j_called) for indel_i_called , SNP_j_called in zip(list(genotypes_for_indel_i == 1) , list(genotypes_for_SNP_j == SNP_j_alt_allele_code))]

                #number of isolates that have both variants
                num_isolates_with_both_variants = np.sum(isolates_with_both_variants_filter)

                #check to see if the number of isolates carrying both variants was >= 1
                if num_isolates_with_both_variants >= 1:

                    # get info for mutation in regulator
                    regulator_INDEL_info = list(regulator_INDELs_DF[regulator_INDELs_DF.key == variant_i].loc[:, ['pos','ref','alt','mut_type','codon_pos','gene_id','gene_name','gene_pos','num_isolates']].values[0])

                    # get info for mutation in regulon
                    regulon_SNP_info = list(regulon_SNPs_DF[(regulon_SNPs_DF.pos == SNP_j_pos) & (regulon_SNPs_DF.alt == SNP_j_alt_allele)].loc[:, ['pos','ref','alt','mut_type','codon_pos','gene_id','gene_name','gene_pos','num_isolates']].values[0])

                    strain_names = [' - '.join(list(INDEL_isolate_annotation_DF[isolates_with_both_variants_filter].isolate_ID))]

                    # sublineage calls
                    strain_lineage_calls_count_dict = Counter(list(INDEL_isolate_annotation_DF[isolates_with_both_variants_filter].lineage_call))
                    strain_lineage_calls_list = []
                    for sublineage_i in strain_lineage_calls_count_dict.keys():
                        sublineage_i_with_mutations = sublineage_i + '({0})'.format(str(strain_lineage_calls_count_dict[sublineage_i]))
                        strain_lineage_calls_list.append(sublineage_i_with_mutations)
                    strain_lineage_calls = ' - '.join(strain_lineage_calls_list)

                    co_occurring_SNP_i_and_SNP_or_indel_j_info = ['INDEL'] + regulator_INDEL_info + ['SNP'] + regulon_SNP_info + [num_isolates_with_both_variants] + strain_names + [strain_lineage_calls]
                    variant_mutation_pair_df.loc[variant_mutation_pair_df_index, :] = co_occurring_SNP_i_and_SNP_or_indel_j_info
                    variant_mutation_pair_df_index += 1

            if variant_j.count('_') == 2: # if variant in regulon is an INDEL

                #genotypes for each INDEL
                genotypes_for_indel_j = INDEL_genotypes_array[INDEL_annotation_DF[INDEL_annotation_DF.key == variant_j].index[0] , :]

                #boolean filter of isolates with both variants
                isolates_with_both_variants_filter = [(indel_i_called and indel_j_called) for indel_i_called , indel_j_called in zip(list(genotypes_for_indel_i == 1) , list(genotypes_for_indel_j == 1))]

                #number of isolates that have both variants
                num_isolates_with_both_variants = np.sum(isolates_with_both_variants_filter)

                #check to see if the number of isolates carrying both variants was >= 1
                if num_isolates_with_both_variants >= 1:

                    # get info for mutation in regulator
                    regulator_INDEL_info = list(regulator_INDELs_DF[regulator_INDELs_DF.key == variant_i].loc[:, ['pos','ref','alt','mut_type','codon_pos','gene_id','gene_name','gene_pos','num_isolates']].values[0])

                    # get info for mutation in regulon
                    regulon_INDEL_info = list(regulon_INDELs_DF[regulon_INDELs_DF.key == variant_j].loc[:, ['pos','ref','alt','mut_type','codon_pos','gene_id','gene_name','gene_pos','num_isolates']].values[0])

                    strain_names = [' - '.join(list(INDEL_isolate_annotation_DF[isolates_with_both_variants_filter].isolate_ID))]

                    # sublineage calls
                    strain_lineage_calls_count_dict = Counter(list(INDEL_isolate_annotation_DF[isolates_with_both_variants_filter].lineage_call))
                    strain_lineage_calls_list = []
                    for sublineage_i in strain_lineage_calls_count_dict.keys():
                        sublineage_i_with_mutations = sublineage_i + '({0})'.format(str(strain_lineage_calls_count_dict[sublineage_i]))
                        strain_lineage_calls_list.append(sublineage_i_with_mutations)
                    strain_lineage_calls = ' - '.join(strain_lineage_calls_list)

                    co_occurring_SNP_i_and_SNP_or_indel_j_info = ['INDEL'] + regulator_INDEL_info + ['INDEL'] + regulon_INDEL_info + [num_isolates_with_both_variants] + strain_names + [strain_lineage_calls]
                    variant_mutation_pair_df.loc[variant_mutation_pair_df_index, :] = co_occurring_SNP_i_and_SNP_or_indel_j_info
                    variant_mutation_pair_df_index += 1

In [100]:
variant_mutation_pair_df

,A_type,A_pos,A_ref,A_alt,A_mut_type,A_codon_pos,A_gene_id,A_gene_name,A_gene_pos,A_num_isolates,...,B_alt,B_mut_type,B_codon_pos,B_gene_id,B_gene_name,B_gene_pos,B_num_isolates,num_isolates_co_occur,isolate_IDs,sublineages


In [101]:
np.shape(variant_mutation_pair_df)

(0, 23)

Save dataframe as CSV file

In [98]:
# variant_mutation_pair_df.to_excel('/n/data1/hms/dbmi/farhat/Roger/mmpR_BDQ_mutant_project/CSV files/co_occurring_mutations_within_isolates/whiB7_promoter_eis_gene_body_co_occurence_mutations_in_31440_isolates.xlsx')